<a href="https://colab.research.google.com/github/JaneOliveira/DataScience_BayesIngenuo/blob/main/Bayes_Ingenuo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# brincando com ML

####Criando bayes ingênuo
---



In [1]:
import numpy as np

In [2]:
class bayes:
  def __init__(self,data) -> None:
    self.dataset = data
    self.n, self.d = np.shape(self.dataset)


  def cria_conj_test(self) -> None:
    self.n_amostras = int((self.dataset.shape[0])*0.2)
    coluna = int((self.dataset.shape[1]))
    np.random.shuffle(self.dataset)
    self.conjunto_teste = np.zeros((self.n_amostras, coluna))
    for i in range(self.n_amostras):
        for j in range(coluna):
            self.conjunto_teste[i,j] = self.dataset[i,j]
    self.deleta_teste()

  
  def deleta_teste(self) ->None:
    for i in range(self.n_amostras):
      self.dataset = np.delete(self.dataset,i, 0)


  def normaliza(self) -> None:
    nInput = self.d - 1
    for l in range(nInput):
      self.dataset[:,l] = 2*(self.dataset[:,l] - self.dataset[:,l].min(axis=0)) / (self.dataset[:,l].max(axis=0) - self.dataset[:,l].min(axis=0)) - 1

  def calcula_media_classe(self) -> None:
    self.mediaClasse = np.zeros(((2,self.dataset.shape[1]-1)))   
    cont2 =0
    cont1 =0
    for i in range((self.dataset.shape[1]-1)):
        for j in range(self.dataset.shape[0]):
            if(self.dataset[j,-1] == 1):
                self.mediaClasse[0][i]+=self.dataset[j,i]
                cont1+=1
            else:
                self.mediaClasse[1][i]+=self.dataset[j,i]
                cont2+=1 
                
        self.mediaClasse[0][i]/=cont1
        self.mediaClasse[1][i]/=cont2

  
  def calcula_variancia_classe(self) -> None:
    self.calcula_media_classe()
    self.varianciaClasse = np.zeros(((2,self.dataset.shape[1]-1))) 
    cont2 =0
    cont1 =0
    
    for i in range(self.dataset.shape[1]-1):
        for j in range((self.dataset.shape[0])):
            if(self.dataset[j,-1] == 1):
                self.varianciaClasse[0][i]+=self.dataset[j,i]
                cont1+=1
            else:
                self.varianciaClasse[1][i]+=self.dataset[j,i]
                cont2+=1               
        self.varianciaClasse[0][i]/=cont1
        self.varianciaClasse[1][i]/=cont2
    
  def calcula_condicional(self,teste) -> None:
    self.calcula_variancia_classe()
    mu1 = self.mediaClasse[0]
    mu2 = self.mediaClasse[1]
    sigma1 = self.varianciaClasse[0]
    sigma2 = self.varianciaClasse[1]
    self.condicionalClasse = np.zeros(((2,self.dataset.shape[1]-1)))
    for i in range((self.dataset.shape[1]-1)):
        self.condicionalClasse[0][i]= (1.0/(sigma1[i] * np.sqrt(2.0 * np.pi))) * np.exp( - ((teste[i] - mu1[i])**2.0) / (2.0 * sigma1[i]**2.0))
        self.condicionalClasse[1][i]= (1.0/(sigma2[i] * np.sqrt(2.0 * np.pi))) * np.exp( - ((teste[i] - mu2[i])**2.0) / (2.0 * sigma2[i]**2.0))

  def calculaPriori(self) -> None:
    self.PrioriClasse = np.zeros((2))
    tam_base = self.dataset.shape[0]
    for i in range(tam_base):
        if(self.dataset[i,-1] == 1):
            self.PrioriClasse[0]+=1
        else:
            self.PrioriClasse[1]+=1
    
    self.PrioriClasse[0]/=tam_base
    self.PrioriClasse[1]/=tam_base

  def fit(self,teste) -> int:
    self.calculaPriori()
    self.calcula_condicional(teste)

    self.probabilidade = np.ones(((2)))
    
    for i in range(self.dataset.shape[1]-1):
        self.probabilidade[0] *= (self.condicionalClasse[0][i])
        self.probabilidade[1] *= (self.condicionalClasse[1][i])

        
    self.probabilidade[0]*= self.PrioriClasse[0]
    self.probabilidade[1]*= self.PrioriClasse[1]
    
    if(self.probabilidade[0] <= self.probabilidade[1]):
        return 1
    else:
        return 0


  def predict(self) -> None:
    tam_conjunto_teste = self.conjunto_teste.shape[0]
    self.acertos =0
    self.erros =0
    
    for i in range(tam_conjunto_teste):
        resultado = self.fit(self.conjunto_teste[i,:])
        if(resultado == self.conjunto_teste[i,-1]):
            self.acertos+=1
        else:
            self.erros+=1
    self.por_acertos = self.acertos*100/(len(self.conjunto_teste))
    self.por_erros = self.erros*100/(len(self.conjunto_teste))
    print("--------------------------")
    print("Resultados classificação bayes ingênuo")
    print("")
    print("--------------------------\n")
    print(f"Acertos:{self.acertos} de um total de {len(self.conjunto_teste)} amostras \nequivale a {self.por_acertos}%")
    print(f"Erros:{self.erros} de um total de {len(self.conjunto_teste)} amostras \nequivale a {self.por_erros}%")

    print("--------------------------")
    
  

In [3]:
def main():
  dataset = np.genfromtxt('base.txt')
  dataset[dataset[:,-1] == 2, -1] = 0  
  model = bayes(dataset)
  model.normaliza()
  model.cria_conj_test()
  model.predict()
main()

--------------------------
Resultados classificação bayes ingênuo

--------------------------

Acertos:143 de um total de 200 amostras 
equivale a 71.5%
Erros:57 de um total de 200 amostras 
equivale a 28.5%
--------------------------
